In [ ]:
%pip install pandas


In [ ]:
import sys
import os
import argparse
import csv
import json
import pandas as pd


In [ ]:
do_example = False
#   - anime : [animeID, (count_reviews), avg_reviews ] 
#   - anime : details [animeID, episodes, ... ]

if do_example is not True :
    #   read current file
    filename = './000_unpolished_csv/animelists_cleaned.csv'
    firaw = pd.read_csv(filename, nrows= 10)
    print(f'head : {firaw.columns.values}')
    #   head : ['username' 'user_id' 'user_watching' 'user_completed' 'user_onhold'
    #            'user_dropped' 'user_plantowatch' 'user_days_spent_watching' 'gender'
    #            'location' 'birth_date' 'access_rank' 'join_date' 'last_online'
    #            'stats_mean_score' 'stats_rewatched' 'stats_episodes']
    
    print(f'{firaw.iloc[0]}')

    f_anime_rews = './final_csv/anime_revs.csv'
    if not os.path.exists(f_anime_rews):
        with open(f_anime_rews, 'w'): pass   

#   copy related only if type is not manga

    head : ['username' 'anime_id' 'my_watched_episodes' 'my_start_date'
     'my_finish_date' 'my_score' 'my_status' 'my_rewatching'
     'my_rewatching_ep' 'my_last_updated' 'my_tags']

    user_id                     !!!!!!!!!!!!!! NOT FOR NOW
    username                          karthiga
    anime_id                                21
    animename                   !!!!!!!!!!!!!! NOT FOR NOW
    my_watched_episodes                    586
    my_score                                 9
    my_status                                1
    my_rewatching                          NaN
    my_rewatching_ep                         0
    my_rev_time                 !!!!!!! _ set emtpy
    my_rev_text                 !!!!!!! _ set emtpy

    Name: 0, dtype: object

In [ ]:
import time
import random

def str_time_prop(start, end, input_time_format, output_time_format, prop):
    stime = time.mktime(time.strptime(start, input_time_format))
    etime = time.mktime(time.strptime(end, input_time_format))
    ptime = stime + prop * (etime - stime)
    return time.strftime(output_time_format, time.localtime(ptime))

def random_date(start, end, prop):
    return str_time_prop(start, end, '%Y-%m-%dT%I:%M:%S', '%Y-%m-%dT%I:%M:%S.000+00:00', prop)
    #   return str_time_prop(start, end, '%Y/%m/%d %I:%M %p', '%Y-%m-%dT%I:%M:%s.000+00:00', prop)

print(random_date("2024-05-01T1:30:05", "2024-06-10T4:50:50", random.random()) )
##  2022-03-24T09:42:41.000+00:00

In [2]:
#   add email, password
import sys
import os
import csv
import json
import pandas as pd
import time
import random
import pickle


##  2022-03-24T09:42:41.000+00:00
def str_time_prop(start, end, input_time_format, output_time_format, prop):
    stime = time.mktime(time.strptime(start, input_time_format))
    etime = time.mktime(time.strptime(end, input_time_format))
    ptime = stime + prop * (etime - stime)
    return time.strftime(output_time_format, time.localtime(ptime))

def random_date(start, end, prop):
    return str_time_prop(start, end, '%Y-%m-%dT%I:%M:%S', '%Y-%m-%dT%I:%M:%S.000+00:00', prop)
    #   return str_time_prop(start, end, '%Y/%m/%d %I:%M %p', prop)



list_anime_revs = [ 
                'username' , 
                'anime_id',
                'my_watched_episodes' , 
                'my_score' , 
                ]


#   DONE :
expanded_list_anime_revs = [ 
                    'user_id'           #  get
                ,   'username'      # have
                ,   'anime_id'      # have
                ,   'animename'         #  get
                ,   'my_score'      # have
                ,   'my_rev_time'       #  rng
                ,   'my_rev_text'    
            ]


list_for_graph = [
        'user_id' 
    ,   'anime_id'
    ,   'my_status'
    ,   'my_watched_episodes'
]


print(f'length of target cols : {len(list_anime_revs)}')



do_merge_reviews = True
if do_merge_reviews == True :

    t0 = time.time()

    # read from source
    src = './000_unpolished_csv/animelists_cleaned.csv'
    file_base = pd.read_csv(src)
    load_time = time.time()

    extracted = file_base[list_anime_revs]
    print(f'length of extr cols : {extracted.columns.values.size} ' )
    print(f'{extracted.iloc[0]}\n\n')
    t1 = time.time()
    print(f'time to load : {t1-t0}')



    #   ADD REVIEW TIME
    ## Date format : 2022-03-24T09:42:41.000+00:00
    extracted['my_rev_time'] = random_date("2024-05-01T1:30:05", "2024-06-10T4:50:50", random.random())
    #   extracted['my_rev_text'] = ''
    print(f'{extracted.iloc[0]} \n\n')
    t2 = time.time()
    print(f'time to rng timestamp : {t2-t1}')



    #   ADD USER_ID based on USERNAME
    usernamefile = './final_csv/userid_username_graph.csv'
    with open(usernamefile, 'rb') as namefile :
        namedf = pd.read_csv(namefile)
    extracted = pd.merge(
                    extracted
                ,   namedf
                ,   on= 'username'
                ,   how = 'left'
    )
    print(f'{extracted.iloc[0]} \n\n')
    t3 = time.time()
    print(f'time to merge username : {t3-t2}')



    #   ADD ANIMENAME based on ANIME_ID
    animefilename = './final_csv/animeid_aniname_graph.csv'
    with open(animefilename, 'rb') as anifile :
        animedf = pd.read_csv(anifile)
    animedf.rename(columns={ 'anime_id':'anime_id' , 'title':'animename' })
    extracted = pd.merge(
                    extracted
                ,   animedf
                ,   on= 'anime_id'
                ,   how = 'left'
    )
    print(f'{extracted.iloc[0]} \n\n')
    t4 = time.time()
    print(f'time to merge anime_id : {t4-t3}')

    

    #   ADD REVIEW TEXT 
    ### username,anime_id,score,text
    reviewfilename = './final_csv/cleaned_score_revs.csv'
    with open(reviewfilename, 'rb') as revfile :
        revdf = pd.read_csv(revfile)
        #   redfil.close()

    #   revdf.rename(columns={ 'username':'username' , 'anime_id':'anime_id', 'score':'my_score' , 'text':'text' })

    #   revindex_vals = ['username', 'anime_id', 'my_score', 'text']
    print(f'len origin : {len(extracted)}')
    print(f'len rev_db : {len(revdf)}')
    
    merged = pd.merge(
                        extracted
                    ,   revdf
                    ,   how='left'
                    ,   on=['username', 'anime_id', 'my_score'] 
                )

    print(f'len merges : {len(merged)}')

    print(f'{merged.iloc[0]} \n\n')
    expand_time = time.time()  
    t5 = time.time()
    print(f'time to merge reviews : {t5-t4}')




    # SAVE to destination

    dst = './final_csv/0final_anime_revs.csv'
    if not os.path.exists(dst):
        with open(dst, 'w'): pass 
    merged.to_csv(dst, index=False)




    save_time = time.time()
    t6 = time.time()
    print(f'time to save : {t6-t5}')
    #   print(extracted.iloc[0])
    print(f'total time : {t6-t0} seconds')

    print(f'\n\n{merged.iloc[0]} \n\n')

    print("DBG -> the end")







length of target cols : 4
length of extr cols : 4 
username               karthiga
anime_id                     21
my_watched_episodes         586
my_score                      9
Name: 0, dtype: object


time to load : 27.00446844100952
username                                    karthiga
anime_id                                          21
my_watched_episodes                              586
my_score                                           9
my_rev_time            2024-05-23T10:52:42.000+00:00
Name: 0, dtype: object 


time to rng timestamp : 0.0760045051574707


C:\Users\Admin\AppData\Local\Temp\ipykernel_11440\3976122693.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extracted['my_rev_time'] = random_date("2024-05-01T1:30:05", "2024-06-10T4:50:50", random.random())


username                                    karthiga
anime_id                                          21
my_watched_episodes                              586
my_score                                           9
my_rev_time            2024-05-23T10:52:42.000+00:00
user_id                                      2255153
Name: 0, dtype: object 


time to merge username : 3.2980005741119385
username                                    karthiga
anime_id                                          21
my_watched_episodes                              586
my_score                                           9
my_rev_time            2024-05-23T10:52:42.000+00:00
user_id                                      2255153
title                                      One Piece
Name: 0, dtype: object 


time to merge anime_id : 3.6919963359832764
username                                    karthiga
anime_id                                          21
my_watched_episodes                              586
my_score    

In [ ]:
filename = './final_csv/anime_revs.csv'
firaw = pd.read_csv(filename, nrows= 10)
print(f'head : {firaw.columns.values}')

print(f'{firaw.iloc[0]}')